In [1]:
import os
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import lxml
import numbers
from bs4 import BeautifulSoup
import requests

In [2]:
headers1 = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [3]:
base_url = "https://www.zillow.com/homes/for_sale/"

In [4]:
url_list = []

city = 'new york city'
url1 = base_url +city+'/'
url_list.append(url1)

for i in range(2,21):
    dom = base_url + city +'/'+str(i)+'_p/'
    url_list.append(dom)

In [5]:
url_list

['https://www.zillow.com/homes/for_sale/new york city/',
 'https://www.zillow.com/homes/for_sale/new york city/2_p/',
 'https://www.zillow.com/homes/for_sale/new york city/3_p/',
 'https://www.zillow.com/homes/for_sale/new york city/4_p/',
 'https://www.zillow.com/homes/for_sale/new york city/5_p/',
 'https://www.zillow.com/homes/for_sale/new york city/6_p/',
 'https://www.zillow.com/homes/for_sale/new york city/7_p/',
 'https://www.zillow.com/homes/for_sale/new york city/8_p/',
 'https://www.zillow.com/homes/for_sale/new york city/9_p/',
 'https://www.zillow.com/homes/for_sale/new york city/10_p/',
 'https://www.zillow.com/homes/for_sale/new york city/11_p/',
 'https://www.zillow.com/homes/for_sale/new york city/12_p/',
 'https://www.zillow.com/homes/for_sale/new york city/13_p/',
 'https://www.zillow.com/homes/for_sale/new york city/14_p/',
 'https://www.zillow.com/homes/for_sale/new york city/15_p/',
 'https://www.zillow.com/homes/for_sale/new york city/16_p/',
 'https://www.zillow.

In [6]:
dom

'https://www.zillow.com/homes/for_sale/new york city/20_p/'

In [7]:
def soups(data):
    with requests.Session() as s:
        r = s.get(data, headers=headers1)
        soup = BeautifulSoup(r.content, 'html.parser')
    return soup

In [8]:
list_links = []

for url in url_list:
    htmls = soups(url)
    list_links.append(htmls)
print(len(list_links))

20


In [9]:
df = pd.DataFrame()

In [10]:
#extract the data we want from the html in our list.

def z_soup(data):
    
    address = data.find_all(class_= 'list-card-addr')
    price = list(data.find_all(class_='list-card-price'))
    beds = list(data.find_all("ul", class_="list-card-details"))
    
     #create dataframe columns out of variables
    df['prices'] = price
    df['address'] = address
    df['beds'] = beds
    
    return df.copy()

In [11]:
zillow_df = pd.DataFrame()

In [12]:
# Get our data from each url and put it into a list of dataframes

df_list = []
for soup in list_links:
    new_df = z_soup(soup)
    df_list.append(new_df)

In [13]:
# Combine the list of datasets into our new dataframe
zillow_df = pd.concat(df_list)
zillow_df.reset_index(inplace=True)
zillow_df = zillow_df.drop('index', axis=1)

In [14]:
print(zillow_df.shape)
zillow_df.head()

(180, 3)


,prices,address,beds
0,"[$466,000]","[121 Tapscott St, Brooklyn, NY 11212]","[[6, [ , , bds]], [2, [ , , ba]], [2,240, [ ..."
1,"[$899,000]","[709 E 51st St, Brooklyn, NY 11203]","[[5, [ , , bds]], [4, [ , , ba]], [3,200, [ ..."
2,"[$675,000]","[933 E 88th St, Brooklyn, NY 11236]","[[4, [ , , bds]], [2, [ , , ba]], [1,855, [ ..."
3,"[$255,000]","[24 Doscher St, Brooklyn, NY 11208]","[[5, [ , , bds]], [2, [ , , ba]], [1,404, [ ..."
4,"[$325,000]","[97 S 8th St APT 22, Brooklyn, NY 11249]","[[2, [ , , bds]], [1, [ , , ba]], [--, [ , ..."


In [15]:
zillow_df = zillow_df.applymap(str)

In [16]:
zillow_df = zillow_df.applymap(lambda x: re.sub('<[^<]+?>', '',x))
zillow_df.head()

,prices,address,beds
0,"$466,000","121 Tapscott St, Brooklyn, NY 11212","6 bds2 ba2,240 sqft- Multi-family home for sale"
1,"$899,000","709 E 51st St, Brooklyn, NY 11203","5 bds4 ba3,200 sqft- House for sale"
2,"$675,000","933 E 88th St, Brooklyn, NY 11236","4 bds2 ba1,855 sqft- House for sale"
3,"$255,000","24 Doscher St, Brooklyn, NY 11208","5 bds2 ba1,404 sqft- House for sale"
4,"$325,000","97 S 8th St APT 22, Brooklyn, NY 11249",2 bds1 ba-- sqft- Condo for sale


In [17]:
zillow_df[['beds', 'home_type']] = zillow_df.beds.str.split('-',n=1, expand=True)
zillow_df.head()

,prices,address,beds,home_type
0,"$466,000","121 Tapscott St, Brooklyn, NY 11212","6 bds2 ba2,240 sqft",Multi-family home for sale
1,"$899,000","709 E 51st St, Brooklyn, NY 11203","5 bds4 ba3,200 sqft",House for sale
2,"$675,000","933 E 88th St, Brooklyn, NY 11236","4 bds2 ba1,855 sqft",House for sale
3,"$255,000","24 Doscher St, Brooklyn, NY 11208","5 bds2 ba1,404 sqft",House for sale
4,"$325,000","97 S 8th St APT 22, Brooklyn, NY 11249",2 bds1 ba,- sqft- Condo for sale


In [18]:
zillow_df[['address', 'borough','zip_code']] = zillow_df.address.str.split(",", expand=True)
zillow_df.head()

,prices,address,beds,home_type,borough,zip_code
0,"$466,000",121 Tapscott St,"6 bds2 ba2,240 sqft",Multi-family home for sale,Brooklyn,NY 11212
1,"$899,000",709 E 51st St,"5 bds4 ba3,200 sqft",House for sale,Brooklyn,NY 11203
2,"$675,000",933 E 88th St,"4 bds2 ba1,855 sqft",House for sale,Brooklyn,NY 11236
3,"$255,000",24 Doscher St,"5 bds2 ba1,404 sqft",House for sale,Brooklyn,NY 11208
4,"$325,000",97 S 8th St APT 22,2 bds1 ba,- sqft- Condo for sale,Brooklyn,NY 11249


In [20]:
# separate beds column into bed, bath, and sq_feet

zillow_df[['beds', 'baths', 'sqft']] = zillow_df.beds.str.split(' ',n=2, expand=True)
zillow_df['sqft'] = zillow_df.sqft.str.replace(",", "")

In [21]:
# extract only the digits from the columns
zillow_df['beds'] = zillow_df.beds.str.extract('(\d+)')
zillow_df['baths'] = zillow_df.baths.str.extract('(\d+)')
zillow_df['sqft'] = zillow_df.sqft.str.extract('(\d+)')

In [22]:
# convert columns to float
zillow_df['beds'] = zillow_df['beds'].astype('float')
zillow_df['baths'] = zillow_df['baths'].astype('float')
zillow_df['sqft'] = zillow_df['sqft'].astype('float')

In [23]:
zillow_df.tail()

,prices,address,beds,home_type,borough,zip_code,baths,sqft
175,"$599,000",230 Jay St #5F,1.0,Condo for sale,Brooklyn,NY 11201,1.0,700.0
176,"$625,000",431 E 32nd St,4.0,House for sale,Brooklyn,NY 11226,2.0,2500.0
177,"$890,000",1 Hanson Pl APT 10A,1.0,Condo for sale,Brooklyn,NY 11243,2.0,824.0
178,"$2,000,000",51 Jay St #4L,2.0,Condo for sale,Brooklyn,NY 11201,2.0,1287.0
179,"$215,000",2212 Brigham St APT 3C,3.0,Condo for sale,Brooklyn,NY 11229,1.0,1100.0
